# YOLO Dataset Augmentation Pipeline - Interactive Demo
## ----------------------------------------------

This notebook demonstrates the comprehensive YOLO dataset augmentation system
with bbox-focused transformations for object detection tasks.

Author: Pritam Thapa
Purpose: Showcase augmentation capabilities and visual results

## YOLO Dataset Augmentation Pipeline Demo

## This notebook demonstrates:
## 1. **Dataset Preparation**: Scanning, converting, repairing labels
## 2. **Bbox-Focused Augmentations**: 8 custom transform types
## 3. **Quality Control**: Validation and filtering
## 4. **Pipeline Automation**: End-to-end workflow
# 
## Key Features
## - Bbox-localized transformations (don't affect background)
## - Sport-specific augmentations (ball detection, motion blur, occlusion)
## - Realistic lighting effects (stadium lights, shadows, reflections)
## - Quality validation (black frame detection, bbox integrity)
## - Automatic target-based generation

## Install dependencies

In [ ]:
!pip install opencv-python numpy pyyaml tqdm albumentations matplotlib

## Setup and Imports

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from pathlib import Path
import random
import yaml
from IPython.display import display, HTML
import warnings
warnings.filterwarnings('ignore')

# Set matplotlib style
plt.style.use('default')
%matplotlib inline

## Helper functions for visualization

In [ ]:
def load_image(path):
    """Load image in RGB format"""
    img = cv2.imread(str(path))
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB) if img is not None else None

def load_yolo_labels(label_path):
    """Load YOLO format labels"""
    bboxes, labels = [], []
    if Path(label_path).exists():
        with open(label_path, 'r') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) >= 5:
                    labels.append(int(parts[0]))
                    bboxes.append([float(x) for x in parts[1:5]])
    return bboxes, labels

def yolo_to_pixels(bbox, img_shape):
    """Convert YOLO format to pixel coordinates"""
    h, w = img_shape[:2]
    x_center, y_center, width, height = bbox
    x1 = int((x_center - width / 2) * w)
    y1 = int((y_center - height / 2) * h)
    x2 = int((x_center + width / 2) * w)
    y2 = int((y_center + height / 2) * h)
    return x1, y1, x2, y2

def draw_bboxes(ax, image, bboxes, color='lime', linewidth=2):
    """Draw bounding boxes on image"""
    ax.imshow(image)
    for bbox in bboxes:
        x1, y1, x2, y2 = yolo_to_pixels(bbox, image.shape)
        rect = Rectangle((x1, y1), x2-x1, y2-y1, 
                        linewidth=linewidth, edgecolor=color, facecolor='none')
        ax.add_patch(rect)
    ax.axis('off')

def visualize_comparison(original, augmented, bboxes, title="Augmentation Result"):
    """Side-by-side comparison of original and augmented images"""
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    draw_bboxes(axes[0], original, bboxes, color='lime')
    axes[0].set_title('Original', fontsize=14, fontweight='bold')
    
    draw_bboxes(axes[1], augmented, bboxes, color='cyan')
    axes[1].set_title(title, fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.show()

def visualize_grid(images, titles, bboxes_list, cols=3):
    """Display multiple augmented versions in a grid"""
    n = len(images)
    rows = (n + cols - 1) // cols
    fig, axes = plt.subplots(rows, cols, figsize=(5*cols, 5*rows))
    axes = axes.flatten() if n > 1 else [axes]
    
    for idx, (img, title, bboxes) in enumerate(zip(images, titles, bboxes_list)):
        draw_bboxes(axes[idx], img, bboxes, color='yellow')
        axes[idx].set_title(title, fontsize=12, fontweight='bold')
    
    # Hide unused subplots
    for idx in range(n, len(axes)):
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.show()

## 1. Dataset Preparation Demo
### Dataset Structure Detection and Conversion

In [ ]:
print("=" * 80)
print("DATASET PREPARATION UTILITIES")
print("=" * 80)

# Example: Scan dataset structure
def demo_scan_dataset(dataset_path):
    """Demonstrate dataset scanning"""
    from src.yolo_augmentor.data.scan_dataset import scan_dataset
    
    print(f"\n Scanning dataset: {dataset_path}")
    result = scan_dataset(dataset_path)
    
    print(f"\n Scan Results:")
    print(f"   Total Images: {result['total_images']}")
    print(f"   Total Labels: {result['total_labels']}")
    print(f"   Missing Pairs: {result['missing_pairs']}")
    print(f"   Structure Type: {result['structure_type']}")
    
    if result['has_problems']:
        print(f"\n  Issues detected:")
        print(f"   First few missing labels: {result['missing_label_files'][:5]}")
    
    return result

# Example usage (uncomment to run on your dataset)
# demo_scan_dataset("path/to/your/dataset")

### Label Repair and Validation

In [ ]:
def demo_label_repair():
    """Demonstrate label repair capabilities"""
    print("\n🔧 LABEL REPAIR CAPABILITIES")
    print("-" * 50)
    print("Automatically fixes:")
    print("  ✓ Malformed coordinate formats")
    print("  ✓ Out-of-range bbox values (clips to [0,1])")
    print("  ✓ Zero/negative bbox dimensions")
    print("  ✓ Invalid class labels")
    print("\nExample fixes:")
    
    examples = [
        ("0 0.5 0.5 1.2 0.8", "0 0.500000 0.500000 1.000000 0.800000", "Clipped width > 1.0"),
        ("0 0.5 0.5 0 0.3", "REMOVED", "Zero width detected"),
        ("0 -0.1 0.5 0.3 0.4", "0 0.000000 0.500000 0.300000 0.400000", "Clipped negative x"),
    ]
    
    for original, fixed, reason in examples:
        print(f"\n  Original: {original}")
        print(f"  Fixed:    {fixed}")
        print(f"  Reason:   {reason}")

demo_label_repair()

## 2. Custom Augmentation Transforms
### This section demonstrates all 8 custom bbox-focused augmentation types.

### Transform Categories:
### 1. **Motion Simulation**: Shear + blur for fast-moving objects
### 2. **Occlusion**: Realistic object hiding/overlap
### 3. **Lighting Effects**: Stadium lights, reflections, shadows
### 4. **Degradation**: Noise, pixel dropout, compression artifacts

In [ ]:
# Create Sample Test Image

def create_sample_image():
    """Create a sample image with a ball-like object for testing"""
    img = np.ones((480, 640, 3), dtype=np.uint8) * 120  # Gray background
    
    # Add some texture to background
    noise = np.random.randint(-20, 20, (480, 640, 3))
    img = np.clip(img + noise, 0, 255).astype(np.uint8)
    
    # Draw a ball (circular object)
    center = (320, 240)
    radius = 40
    cv2.circle(img, center, radius, (255, 200, 50), -1)  # Yellow ball
    cv2.circle(img, center, radius, (200, 150, 30), 3)   # Border
    
    # Add highlight
    cv2.circle(img, (center[0]-10, center[1]-10), 12, (255, 255, 200), -1)
    
    # YOLO format bbox for the ball
    x_center = center[0] / 640
    y_center = center[1] / 480
    width = (radius * 2) / 640
    height = (radius * 2) / 480
    bbox = [x_center, y_center, width, height]
    
    return img, [bbox], [0]

# Create sample
sample_img, sample_bboxes, sample_labels = create_sample_image()

fig, ax = plt.subplots(1, 1, figsize=(8, 6))
draw_bboxes(ax, sample_img, sample_bboxes, color='lime')
ax.set_title('Sample Test Image with Bbox', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 3. Augmentation Showcase

### Transform 1: BboxMultiBlurAndShearTransform
### **Purpose**: Simulate fast-moving objects (e.g., pickle ball in flight)

### **Effects**:
### - Random blur types (Gaussian, Median, Bilateral, Defocus)
### - Aggressive shear transformation
### - Motion trail simulation

In [ ]:
import sys
import os

# Get the path to your project's root directory (adjust path as necessary)
# For example, if the notebook is two levels deep from the root:
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..')) 
if project_root not in sys.path:
    sys.path.append(project_root)

In [ ]:
print("Current Working Directory (Your Root):", os.getcwd())
print("Files in this directory:", os.listdir(os.getcwd()))

In [ ]:
from src.yolo_augmentor.aug.custom_transforms import BboxMultiBlurAndShearTransform

print("\n🎯 Transform 1: Bbox Multi-Blur + Shear")
print("-" * 60)
print("Use Case: Fast-moving ball detection")
print("Effects: Motion blur + directional shear")

# Create multiple variations
transform = BboxMultiBlurAndShearTransform(
    blur_types=['gaussian', 'median', 'defocus'],
    blur_strength_range=(5, 15),
    shear_x_range=(-80, 80),
    shear_y_range=(-40, 40),
    bbox_prob=1.0
)

augmented_images = []
titles = []

for i in range(6):
    aug_img, aug_boxes, aug_labels = transform.apply(
        sample_img.copy(), sample_bboxes.copy(), sample_labels.copy()
    )
    augmented_images.append(aug_img)
    titles.append(f'Variation {i+1}')

visualize_grid([sample_img] + augmented_images[:5], 
               ['Original'] + titles[:5],
               [sample_bboxes] * 6,
               cols=3)

## Transform 2: BboxExtremeShearOcclude
 
### **Purpose**: Extreme motion + occlusion + brightness changes
 
### **Effects**:
### - Extreme shear angles
### - Brightness multiplication
### - Random occlusion patterns

In [ ]:
from src.yolo_augmentor.aug.custom_transforms import BboxExtremeShearOcclude

print("\n🎯 Transform 2: Extreme Shear + Occlusion + Brightness")
print("-" * 60)
print("Use Case: Challenging detection scenarios")
print("Effects: Extreme motion + lighting + occlusion")

transform = BboxExtremeShearOcclude(
    shear_x_range=(-100, 100),
    shear_y_range=(-60, 60),
    brightness_shift=(1.5, 2.5),
    occlusion_intensity='medium',
    bbox_prob=1.0
)

augmented_images = []
for i in range(5):
    aug_img, _, _ = transform.apply(
        sample_img.copy(), sample_bboxes.copy(), sample_labels.copy()
    )
    augmented_images.append(aug_img)

visualize_grid([sample_img] + augmented_images, 
               ['Original'] + [f'Extreme {i+1}' for i in range(5)],
               [sample_bboxes] * 6,
               cols=3)

## Transform 3: NearBboxExtremeBrighten

### **Purpose**: Stadium lighting effects, flash reflections
 
### **Effects**:
### - Rectangular halo around bbox (not inside)
### - Asymmetric expansion
### - Color temperature variations (warm/cool/white)
### - Edge-only illumination

In [ ]:
from src.yolo_augmentor.aug.custom_transforms import NearBboxExtremeBrighten

print("\n🎯 Transform 3: Near-Bbox Extreme Brightening")
print("-" * 60)
print("Use Case: Stadium lights, camera flash, directional lighting")
print("Effects: Bright halos around objects, not inside")

transform = NearBboxExtremeBrighten(
    expand_horizontal_range=(0.3, 2.0),
    expand_vertical_range=(0.3, 2.0),
    intensity_range=(1.4, 2.5),
    decay='gaussian',
    color_bias='auto',
    edge_only_prob=0.3,
    bbox_prob=1.0
)

augmented_images = []
titles = []

for i in range(5):
    aug_img, _, _ = transform.apply(
        sample_img.copy(), sample_bboxes.copy(), sample_labels.copy()
    )
    augmented_images.append(aug_img)
    titles.append(f'Lighting {i+1}')

visualize_grid([sample_img] + augmented_images, 
               ['Original'] + titles,
               [sample_bboxes] * 6,
               cols=3)

## Transform 4: ConcentratedNoiseTransform

### **Purpose**: Sensor noise, low-light conditions
### 
### **Effects**:
### - Intense Gaussian noise at bbox center
### - Radial falloff to background
### - Simulates camera noise under challenging conditions

In [ ]:
from src.yolo_augmentor.aug.custom_transforms import ConcentratedNoiseTransform

print("\n🎯 Transform 4: Concentrated Noise")
print("-" * 60)
print("Use Case: Low-light photography, high ISO noise")
print("Effects: Strong noise at object center, fading outward")

transform = ConcentratedNoiseTransform(
    sigma_center=(25, 60),
    sigma_background=(3, 15),
    bbox_prob=1.0
)

augmented_images = []
for i in range(5):
    aug_img, _, _ = transform.apply(
        sample_img.copy(), sample_bboxes.copy(), sample_labels.copy()
    )
    augmented_images.append(aug_img)

visualize_grid([sample_img] + augmented_images, 
               ['Original'] + [f'Noise {i+1}' for i in range(5)],
               [sample_bboxes] * 6,
               cols=3)

## Transform 5: BallBlendAndShapeBiasTransform

### **Purpose**: Shape deformation and background blending
 
### **Effects**:
### - Non-uniform scaling (squash/stretch)
### - Background color blending
### - Simulates camera distortion or motion compression

In [ ]:
from src.yolo_augmentor.aug.custom_transforms import BallBlendAndShapeBiasTransform

print("\n🎯 Transform 5: Ball Blend + Shape Bias")
print("-" * 60)
print("Use Case: Perspective distortion, rolling motion")
print("Effects: Shape warping + background color blending")

transform = BallBlendAndShapeBiasTransform(
    warp_strength=(0.05, 0.25),
    bbox_prob=1.0
)

augmented_images = []
for i in range(5):
    aug_img, _, _ = transform.apply(
        sample_img.copy(), sample_bboxes.copy(), sample_labels.copy()
    )
    augmented_images.append(aug_img)

visualize_grid([sample_img] + augmented_images, 
               ['Original'] + [f'Warp {i+1}' for i in range(5)],
               [sample_bboxes] * 6,
               cols=3)

## Transform 6: BallPixelLevelOcclusion

### **Purpose**: Micro-occlusions, compression artifacts
 
### **Effects**:
### - Random pixel dropout
### - Salt-and-pepper noise effect
### - Simulates video compression or transmission errors

In [ ]:
from src.yolo_augmentor.aug.custom_transforms import BallPixelLevelOcclusion

print("\n🎯 Transform 6: Ball Pixel-Level Occlusion")
print("-" * 60)
print("Use Case: Video compression artifacts, transmission errors")
print("Effects: Random pixel dropout with noise")

transform = BallPixelLevelOcclusion(
    dropout_frac=(0.02, 0.25),
    bbox_prob=1.0
)

augmented_images = []
for i in range(5):
    aug_img, _, _ = transform.apply(
        sample_img.copy(), sample_bboxes.copy(), sample_labels.copy()
    )
    augmented_images.append(aug_img)

visualize_grid([sample_img] + augmented_images, 
               ['Original'] + [f'Dropout {i+1}' for i in range(5)],
               [sample_bboxes] * 6,
               cols=3)

## Transform 7: GradientPatchTransform
 
### **Purpose**: Smooth directional lighting effects
 
### **Effects**:
### - Rectangular gradient overlays
### - Directional vs. omnidirectional
### - Color temperature shifts
### - Simulates natural/artificial light sources

In [ ]:
from src.yolo_augmentor.aug.custom_transforms import GradientPatchTransform

print("\n🎯 Transform 7: Gradient Patch")
print("-" * 60)
print("Use Case: Directional lighting, shadows, reflections")
print("Effects: Smooth rectangular gradients around bbox")

transform = GradientPatchTransform(
    expand_horizontal_range=(0.5, 1.8),
    expand_vertical_range=(0.5, 1.8),
    intensity_range=(0.6, 2.0),
    color_shift_scale=0.3,
    blend_profile='gaussian',
    directional_prob=0.4,
    bbox_prob=1.0
)

augmented_images = []
for i in range(5):
    aug_img, _, _ = transform.apply(
        sample_img.copy(), sample_bboxes.copy(), sample_labels.copy()
    )
    augmented_images.append(aug_img)

visualize_grid([sample_img] + augmented_images, 
               ['Original'] + [f'Gradient {i+1}' for i in range(5)],
               [sample_bboxes] * 6,
               cols=3)

## Transform 8: BboxGaussianOccludeShearTransform
 
### **Purpose**: Fog-like occlusion with motion
 
### **Effects**:
### - Gaussian-shaped occlusion masks
### - Semi-transparent overlays
### - Optional shear and blur
### - Simulates fog, haze, or atmospheric effects

In [ ]:
from src.yolo_augmentor.aug.custom_transforms import BboxGaussianOccludeShearTransform

print("\n🎯 Transform 8: Gaussian Occlude + Shear")
print("-" * 60)
print("Use Case: Fog, haze, partial transparency under motion")
print("Effects: Smooth Gaussian occlusion + optional shear")

transform = BboxGaussianOccludeShearTransform(
    num_patches=(1, 3),
    intensity_range=(0.5, 1.5),
    shear_x_range=(-50, 50),
    shear_y_range=(-20, 20),
    color_mode='background',
    blur_after=True,
    bbox_prob=1.0
)

augmented_images = []
for i in range(5):
    aug_img, _, _ = transform.apply(
        sample_img.copy(), sample_bboxes.copy(), sample_labels.copy()
    )
    augmented_images.append(aug_img)

visualize_grid([sample_img] + augmented_images, 
               ['Original'] + [f'Fog {i+1}' for i in range(5)],
               [sample_bboxes] * 6,
               cols=3)

## 4. Quality Control Demonstration
 
### The pipeline includes automatic quality validation:
### - Black frame detection
### - Extreme brightness filtering
### - Bbox integrity validation
### - Transform failure handling

In [ ]:
print("\n🔍 QUALITY CONTROL FEATURES")
print("=" * 60)

qc_features = {
    "Black Frame Detection": {
        "threshold": "90% dark pixels",
        "action": "Discard augmentation"
    },
    "Brightness Validation": {
        "range": "[3, 252] mean brightness",
        "action": "Reject extreme values"
    },
    "Bbox Integrity": {
        "checks": "Size, coordinates, boundaries",
        "action": "Filter invalid boxes"
    },
    "Transform Failures": {
        "handling": "Graceful fallback",
        "logging": "Detailed error tracking"
    },
    "Post-Augmentation": {
        "random_discard": "5-10% target rate",
        "purpose": "Prevent overfitting"
    }
}

for feature, details in qc_features.items():
    print(f"\n✓ {feature}:")
    for key, value in details.items():
        print(f"  • {key.title()}: {value}")

## 5. Configuration System

### The pipeline uses YAML configuration for flexible control.

In [ ]:
print("\n⚙️ CONFIGURATION EXAMPLE")
print("=" * 60)

example_config = """
dataset:
  input_images_dir: "dataset/images"
  input_labels_dir: "dataset/labels"
  output_images_dir: "dataset_aug/images"
  output_labels_dir: "dataset_aug/labels"
  target_total_images: 200

quality_control:
  black_frame_threshold: 0.90
  min_brightness: 3
  max_brightness: 252
  target_discard_rate: 0.08

validation:
  min_bbox_width: 0.01
  min_bbox_height: 0.01
  min_visibility: 0.3
  coord_tolerance: 0.02

augment_passes:
  - name: "bbox_multi_blur_shear"
    type: "custom"
    weight: 1.5
    pipeline:
      - transform: "BboxMultiBlurAndShearTransform"
        params:
          blur_types: ["gaussian", "median", "defocus"]
          blur_strength_range: [5, 15]
          shear_x_range: [-80, 80]
          bbox_prob: 0.8
  
  - name: "near_bbox_extreme_bright"
    type: "custom"
    weight: 0.8
    pipeline:
      - transform: "NearBboxExtremeBrighten"
        params:
          intensity_range: [1.4, 2.5]
          edge_only_prob: 0.3
"""
print(example_config)


## 6. Pipeline Statistics

### Based on the provided log file, here's what the pipeline achieved:

In [ ]:
def display_pipeline_stats():
    """Display statistics from actual pipeline run"""
    stats = {
        "Input Dataset": {
            "Original Images": 14,
            "Labels Found": "14/14 (100%)"
        },
        "Target Configuration": {
            "Target Total": 200,
            "Required Generation": 186
        },
        "Augmentation Passes": {
            "bbox_multi_blur_shear": 39,
            "bbox_extreme_shear_bright_occlude": 33,
            "near_bbox_extreme_bright": 16,
            "concentrated_noise_center": 17,
            "ball_blend_shape_bias": 19,
            "ball_pixel_occlusion": 22,
            "gradient_patches_near_bbox": 10,
            "gaussian_occlude_shear": 28
        },
        "Final Results": {
            "Generated Images": 184,
            "Random Discard": 15,
            "Final Total": 198,
            "Success Rate": "100% (0 failures)"
        },
        "Quality Metrics": {
            "Black Frames": 0,
            "Extreme Brightness": 0,
            "Invalid Bboxes": 0,
            "Transform Failures": 0,
            "Overall Discard Rate": "0.0%"
        }
    }
    
    print("\n📊 PIPELINE EXECUTION STATISTICS")
    print("=" * 60)
    
    for section, data in stats.items():
        print(f"\n{section}:")
        if isinstance(data, dict):
            for key, value in data.items():
                print(f"  • {key}: {value}")
        else:
            print(f"  {data}")

display_pipeline_stats()

## 7. Complete Pipeline Usage

### Full Automation Example

In [ ]:
print("\n🚀 COMPLETE PIPELINE WORKFLOW")
print("=" * 60)

pipeline_steps = """
# Step 1: Scan Dataset
yolo-forge scan --path dataset/

# Step 2: Convert to YOLO Format (if needed)
yolo-forge convert --input dataset_raw/ --output dataset_yolo/

# Step 3: Repair Labels
yolo-forge repair --labels dataset_yolo/labels/

# Step 4: Run Augmentation
yolo-forge augment --config config_aug.yaml

# Step 5: Split Train/Val/Test
yolo-forge split --input dataset_aug/ --output dataset_final/ \\
                 --train 0.8 --val 0.1 --test 0.1

# OR: Run Complete Pipeline
yolo-forge pipeline --config pipeline_config.yaml
"""

print(pipeline_steps)

## 8. Best Practices and Tips

In [ ]:
print("\n💡 BEST PRACTICES")
print("=" * 60)

tips = {
    "Augmentation Intensity": [
        "Start with conservative parameters",
        "Gradually increase based on validation performance",
        "Monitor discard rates (aim for <10%)"
    ],
    "Quality Control": [
        "Black frame threshold: 0.85-0.95 (not too strict)",
        "Brightness range: [5, 250] is safer than [3, 252]",
        "Always enable bbox validation"
    ],
    "Transform Selection": [
        "Use motion transforms for sports/action",
        "Use lighting transforms for indoor/outdoor variation",
        "Use occlusion for crowded scenes"
    ],
    "Target Settings": [
        "Set realistic target_total_images",
        "Use weights to balance transform distribution",
        "Enable random post-discard (5-8%)"
    ],
    "Performance": [
        "Process in batches for large datasets",
        "Enable logging for debugging",
        "Backup original data before pipeline"
    ]
}

for category, items in tips.items():
    print(f"\n✓ {category}:")
    for item in items:
        print(f"  • {item}")

## 9. Real-World Use Cases

In [ ]:
print("\n🎯 REAL-WORLD APPLICATIONS")
print("=" * 60)

use_cases = {
    "Sports Analytics": {
        "objects": "Balls, players, equipment",
        "challenges": "Fast motion, blur, occlusion",
        "recommended": ["BboxMultiBlurAndShear", "AdaptiveMotionBlur", 
                       "BboxOcclusion"]
    },
    "Autonomous Vehicles": {
        "objects": "Vehicles, pedestrians, signs",
        "challenges": "Weather, lighting, distance",
        "recommended": ["ConcentratedNoise", "NearBboxExtremeBrighten",
                       "GradientPatch"]
    },
    "Retail/Inventory": {
        "objects": "Products on shelves",
        "challenges": "Occlusion, varied lighting, angles",
        "recommended": ["BboxOcclusion", "BboxColorJitter",
                       "BallBlendShape"]
    },
    "Medical Imaging": {
        "objects": "Lesions, organs, instruments",
        "challenges": "Low contrast, noise, artifacts",
        "recommended": ["ConcentratedNoise", "BallPixelOcclusion",
                       "GaussianOcclude"]
    }
}

for domain, info in use_cases.items():
    print(f"\n{domain}:")
    print(f"  Objects: {info['objects']}")
    print(f"  Challenges: {info['challenges']}")
    print(f"  Recommended Transforms:")
    for transform in info['recommended']:
        print(f"    - {transform}")

## 10. Summary and Key Takeaways

In [ ]:
print("\n KEY TAKEAWAYS")
print("=" * 60)

summary = """
- Comprehensive Pipeline:
   • End-to-end automation from raw data to train-ready splits
   • Built-in quality control and validation
   • Detailed logging and statistics

- Bbox-Focused Augmentations:
   • 8 custom transforms designed for object detection
   • Localized effects (don't destroy background context)
   • Sport/action-specific augmentations

- Production-Ready:
   • Robust error handling and graceful degradation
   • Configurable via YAML (no code changes needed)
   • Extensive validation and sanity checks

- Proven Results:
   • 100% success rate on test dataset
   • 0% discard rate (all augmentations valid)
   • Near-perfect target achievement (198 vs 200)

- Flexible Architecture:
   • Modular transform system
   • Easy to add new transforms
   • CLI + Python API support
"""

print(summary)